In [5]:
import subprocess
import pandas as pd
import numpy as np

# parameters (change as needed)
mjd_start_values = [59000]
mjd_end_values = [59001]
nspan_values = [300]
ncoeffs_values = [3]
maxha_values = [12]
telescope_code_values = ['GMRT']
frequencies = [320, 321, 322, 323, 324]

# generate input files for different parameter combinations
with open("output_file.txt", 'w') as file:
    for mjd_start in mjd_start_values:
        for mjd_end in mjd_end_values:
            for nspan in nspan_values:
                for ncoeffs in ncoeffs_values:
                    for maxha in maxha_values:
                        for telescope_code in telescope_code_values:
                            for frequency in frequencies: 
                                output_file = f"tempo2_input_{mjd_start}_{mjd_end}_{nspan}_{ncoeffs}_{maxha}_{telescope_code}_{frequency}.txt"
                                parameter_values = [mjd_start, mjd_end, nspan, ncoeffs, maxha, telescope_code, frequency]
                                line = " ".join(map(str, parameter_values)) + "\n"
                                file.write(line)

#AWK commands

#To capture the paramaters from the parameter sweep file and run tempo2 polyco on it 
with open("tempo_param_capture.awk", 'r') as awk_file:
    awk_command = awk_file.read()
subprocess.run(f'awk \'{awk_command}\' output_file.txt | sh', shell=True)

#To extract the coefficients and required constants from output of tempo2 to calculate phase and frequency 
with open("fsweep_extract_coeff.awk", 'r') as awk_file:
    awk_command = awk_file.read()
subprocess.run(f'awk \'{awk_command}\' *polyco_new.dat | sh', shell=True)

#Calculate phae and frequency 

def calculate_phase_and_frequency(mjd, tmid, rphase, coeff1, coeff2, coeff3,f0):
    # Calculate the time difference in minutes
    dt = (mjd - tmid) * 1440

    # Calculate phase
    phase = rphase + dt * 60 * f0 + coeff1 + dt * coeff2 + dt**2 * coeff3
    
    norm_phase  = phase% 1
    # Calculate frequency in Hz
    freq = f0+ (1 / 60) * (coeff2 + 2 * dt * coeff3)

    return phase, freq,norm_phase

df=[]
#freq_sweep = [320,322,324,326,330,332,340,380,400]
mjd_input = 57400.48807695310369326  # Replace with the desired MJD value
#f0 = 11.188233045354
output_pred = []


for frequ in range(len(frequencies)):
    file_path = f'coeff_data_{frequencies[frequ]}.dat'
    df = pd.read_csv(file_path, delim_whitespace=True, comment='#', names=['Pulsar', 'TMID', 'F0','RPHASE', 'COEFF1', 'COEFF2', 'COEFF3'],skiprows=1)

    for index, row in df.iterrows():
        psrname, tmid, f0,rphase, coeff1, coeff2, coeff3 = row
        phase,freq,norm_phase = calculate_phase_and_frequency(mjd_input, tmid, rphase, coeff1, coeff2, coeff3,f0)
        output = [psrname,frequencies[frequ],tmid,mjd_input,phase,norm_phase,freq]
        output_pred.append(output)
        print(f"Pulsar: {psrname}, Freq = {frequencies[frequ]}, MJD: {mjd_input}, Phase: {phase}, Frequency: {freq} Hz")



This program comes with ABSOLUTELY NO WARRANTY.
This is free software, and you are welcome to redistribute it
under conditions of GPL license.

************************************************* 
************************************************* 
[tempo2Util.C:396] Warning: [CLK3] no clock corrections available for clock UTC(gmrt) for MJD 59000.2
[tempo2Util.C:396] Warning: [CLK4] Trying assuming UTC = UTC(gmrt)
[tempo2Util.C:396] Warning: [CLK9] ... ok, using stated approximation 
[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.
[tempo2Util.C:396] Warning: [CLK6] Proceeding assuming UTC =  UTC(gmrt)
This program comes with ABSOLUTELY NO WARRANTY.
This is free software, and you are welcome to redistribute it
under conditions of GPL license.

************************************************* 
************************************************* 
[tempo2Util.C:396] Warning: [CLK3] no clock corrections available for clock UTC(gmrt) for MJD 59000.2
[tempo2Util.C:396